In [1]:
import pandas as pd    # to handle the dataframe
pd.set_option('expand_frame_repr', False) #to avoid the multi-lines formatting of the dataframe
import numpy as np     # to handle numbers and ndarray
import missingno as msno #to visualize missing data and get a quick visual summary of the completeness of the data

In [ ]:
df = pd.read_csv('CSVFileName.csv')


Missing Values

In [ ]:
print('Number of missing values per column')
for colm in df.columns:
  n_mv=df[colm].isnull().sum()
  print(f'{colm}: {n_mv}')

df[colm].isnull() returns how many instances are null in the given column(colm)

Visulasation of Missing values

In [ ]:
msno.bar(df)

Using Barplots

In [ ]:
msno.matrix(df)

Allows to visualize the absence and missing values using a matrix

**Explanation of Missing values** : These Gaphs may help you in building a feedback on why data are mising

In [ ]:
msno.heatmap(df)

In [ ]:
msno.dendrogram(df)

Deleting missing values

In [ ]:
#Deletions
#Listwise deleltions
df_wmv
df_wmv.dropna(subset=['Column'],how='any',inplace=True)
f_wmv['Column'].isnull().sum() # A call to verify if there is still null values or not

Deletes all the null instances with null value in the given **Column**

In [ ]:
#Deletions
#Dropping the entire columns
df_wmv.drop('Column',axis=1,inplace=True)
df_wmv

This action deletes the whole **Column** from the dataset that's why it is not recommanded to use it unless the number of missing values in that data set is **>80%**

In [ ]:
#Imputations for non time series data
# imputing with a constant
from sklearn.impute import SimpleImputer
df_const = df.copy()
#setting strategy to 'constant'
const_imputer = SimpleImputer(strategy='constant') # imputing using constant value
df_const.iloc[:,:] = const_imputer.fit_transform(df_const)
df_const.isnull().sum()

This methode replaces all null values in the whole dataset with a constant:
Usually we don't use this method as it doesn't give any advantage when dealinmg with missing data it only says the missing data are here

In [ ]:
#Imputations for non time series data
# imputing with general central tendency
df_centTend = df.copy()
#setting strategy to 'most_frequent' to impute by the mode
md_imputer = SimpleImputer(strategy='most_frequent')# strategy can also be mean or median, by default mean
df_centTend.iloc[:,:] = md_imputer.fit_transform(df_centTend)
df_centTend.isnull().sum() # TO CHECK IF WE MADE THE RIGGHT WORK OR NOT

This is the most basic sloution as it allows to handel data and give them real results that can be used later inm interpretations ***(Mean dOESN'T WORK ON CATEGORICAL DATA)***

*****Using KNN technique (Advanced Technique)*****

In [ ]:
#Advanced imputation techniques
#Using KNN
num_split=df.loc[:,df.dtypes!=object] # This allows us to extract onlt the numerical columns (object stands for string which we will be ignoring in order to complete KNN data cleaning)
num_split_knn= num_split.copy()
from sklearn.impute import KNNImputer
imputer = KNNImputer(n_neighbors=3, metric='nan_euclidean', weights='uniform') #By default parameters except the n_neighbors changed to 3 as it was 5
num_split_knn.iloc[:,:] = imputer.fit_transform(num_split_knn)
num_split_knn.isnull().sum()

to fill out the missing values we use clustering in order to minimaize the frequencey of using mean or median. it allows for more data diversity and helps in having better data quilty

*****Using MICE (Advanced Technique)*****

In [ ]:
#Advanced imputation techniques
#Using MICE
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer #Multivariate imputer that estimates each feature from all the others, as a function of other features in a round-robin fashion
df_mice = df.copy()

mice_imputer = IterativeImputer()
df_mice['Column'] = mice_imputer.fit_transform(df_mice[['Column']])
df_mice.isnull().sum()

The Column must be numerical and you can do this technique for how many number of numerical columns you want just replace **Column** with your wanted **Column from the dataframe**

**Dealing With Noise and Outliers**

In [ ]:
#Visualization of data curves of the numerical attributes
for column in df_centTend.columns:
  df_centTend[column].plot()
  plt.show()

In [ ]:
#Visualization with scatter plots
df_centTend.plot.scatter(x='Column1', y='Column2')

In [ ]:
#Visualization of scatter plot for all the columns
pd.plotting.scatter_matrix(df_centTend, alpha=0.2)

In [ ]:
##Visualization with the best fit ("trend" line) for a guided understanding and explanation
#plot the data points
df_centTend.plot.scatter(x='Column1', y='Column2', color='DarkBlue')
#Fit a line to the data
x=np.array(df_centTend['Column1'])
y=np.array(df_centTend['Column2'])
a, b = np.polyfit(x, y, 1) #calculating the slope a and the y_intercept b of the line
plt.plot(x, a*x + b, color='red', label='Line of Best Fit') #a linear regression line

In [ ]:
#heatmaps, correlation study and noise
sns.heatmap(df_centTend)

In [ ]:
#heatmaps, correlation study and noise
sns.heatmap(df_centTend.corr(), annot=True)

In [ ]:
#Smoothing noisy data with bining
#Smoothing by bin mean
import random
import statistics
from scipy.stats import binned_statistic

num_bins = n
# Use binned_statistic to calculate mean within each bin
result = binned_statistic(df_centTend['Column'], df_centTend['Column'], bins=num_bins, statistic='mean') #statistic is by default mean, you can choose median to perform smoothing by bin median, or min, max to smooth bin boundaries

# Extract bin edges and binned mean from the result
bin_edges = result.bin_edges
bin_means = result.statistic
bin_indices= result.binnumber
hist=np.bincount(bin_indices)

print('Bin Edges: ', bin_edges)
print('Binned Mean: ', bin_means)
print('Bin Indices: ', hist)

plt.hist(bin_edges, weights=hist)

n : You need  to specify the number of bins (recomandeeed between 5 and 20)

Column : a numerical column that we want to check the noise and outliers in

In [ ]:
#Smoothing noisy data with bining
#Equal-width Binning
# Define the number of bins
num_bins = 3
# Use numpy's histogram function for equal width bins
hist, bins = np.histogram(df_centTend['GStat'], bins=num_bins)
print('Bin Edges: ', bins)
print('Histogram Counts: ', hist)
plt.stairs(hist, bins, fill=True)

This is another type of binning called **Equal-width Binning**

In [ ]:
#Smoothing the data with lowess smoothing
import statsmodels.api as sm
x=np.array(df_centTend['Column1'])
y=np.array(df_centTend['Column2'])
lowess = sm.nonparametric.lowess(y, x, frac=0.3)
plt.scatter(x, y)
plt.plot(lowess[:, 0], lowess[:, 1], c='r')
plt.show()

**Smoothing data with clustering**

In [ ]:
#Smoothing noisy data with clustering
#Importing additional libraries
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

#Transform the data with Principal components analysis (dimensionality reduction technique) before applying Kmeans
pca = PCA(2) #Only keep two components
df = pca.fit_transform(df_centTend.loc[:, ['COLUMN1','...COLUMN_N']])
df.shape

**['COLUMN1','...COLUMN_N']** : ALL of them must be numerical columns

In [ ]:
#Instanciating kmeans with a given number of clusters
kmeans = KMeans(n_clusters= 3)

#predict the labels of clusters.
label = kmeans.fit_predict(df)

print(label)

Making the clusters predection

In [ ]:
#Plotting the clusters
#Getting unique labels
u_labels = np.unique(label)
#plotting the results:
for i in u_labels:
    plt.scatter(df[label == i , 0] , df[label == i , 1] , label = i)
plt.legend()
plt.show()

In [ ]:
#Getting the Centroids
centroids = kmeans.cluster_centers_
u_labels = np.unique(label)

#plotting the results:

for i in u_labels:
    plt.scatter(df[label == i , 0] , df[label == i , 1] , label = i)
plt.scatter(centroids[:,0] , centroids[:,1] , s = 80, color = 'k')
plt.legend()
plt.show()

Shows the centroids and the whole data in a scatterplot to better help in understanding where the mean values for each class are located